In [35]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response = llm.invoke("The first person to land on earth was ...")
print(response.content)

I think there might be some confusion.  The first person to land on Earth would be the first human to be born on the planet. However, I believe you might be referring to the first person to walk on the Moon or another celestial body.

If that's the case, the first person to walk on the Moon was Neil Armstrong, an American astronaut who landed on July 20, 1969, as part of the Apollo 11 mission.

If you could provide more context or clarify your question, I'd be happy to try and assist you further.


In [36]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2825619/front-end-engineer-2025-can")
page_data = loader.load().pop().page_content
print(page_data)

Front-End Engineer – 2025 (CAN) - Job ID: 2825619 | Amazon.jobs
Skip to main contentHomeYour job applicationAmazon culture & benefitsDiversity at AmazonLocationsTeamsJob categoriesResourcesInterview tipsDisability accommodationsAbout AmazonFAQ×Front-End Engineer – 2025 (CAN)Job ID: 2825619 | Amazon Development Centre Canada ULC - K03Apply nowDESCRIPTIONAre you excited to build products that are used by Amazon customers all over the world? As a Front-End Engineer, you’ll have the opportunity to work closely with experienced developers to create the user-facing side of software. You’ll collaborate with team members and stakeholders on UX and the full software development lifecycle, including scoping, software design, code, testing, documentation, deployment, and maintenance. NOTE: By applying to this position, your application will be considered for all locations we hire for in Canada including, but not limited to: Vancouver, BC; Toronto, ON, Winnipeg, MN, Victoria, BC, Ottawa, ON, Calga

In [37]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [38]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Front-End Engineer',
 'experience': 'Currently working towards a Bachelor’s or Master’s Degree in Computer Science, Computer Engineering, Information Science, Web Development, Human-Computer Interaction, or other equivalent discipline, with an expected conferral date between September 2024 – September 2025 and/or completed your degree no more than 24 months ago',
 'skills': ['HTML',
  'CSS',
  'JavaScript',
  'Object-oriented programming language (e.g., JavaScript, Python, C#)',
  'Computer science fundamentals (e.g., data structures & algorithms, OO design)',
  'Front-end frameworks such as React, Angular',
  'UX best practices'],
 'description': 'As a Front-End Engineer, you’ll have the opportunity to work closely with experienced developers to create the user-facing side of software. You’ll collaborate with team members and stakeholders on UX and the full software development lifecycle, including scoping, software design, code, testing, documentation, deployment, and maint

In [39]:
type(json_res)

dict

In [40]:
import pandas as pd

df = pd.read_csv("portfolio_links.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://drive.google.com/file/d/sample-pdf-1/view
1,"Angular, .NET, SQL Server",https://drive.google.com/file/d/sample-pdf-2/view
2,"Vue.js, Ruby on Rails, PostgreSQL",https://drive.google.com/file/d/sample-pdf-3/view
3,"Python, Django, MySQL",https://drive.google.com/file/d/sample-pdf-4/view
4,"Java, Spring Boot, Oracle",https://drive.google.com/file/d/sample-pdf-5/view
5,"Flutter, Firebase, GraphQL",https://drive.google.com/file/d/sample-pdf-6/view
6,"WordPress, PHP, MySQL",https://drive.google.com/file/d/sample-pdf-7/view
7,"Magento, PHP, MySQL",https://drive.google.com/file/d/sample-pdf-8/view
8,"React Native, Node.js, MongoDB",https://drive.google.com/file/d/sample-pdf-9/view
9,"iOS, Swift, Core Data",https://drive.google.com/file/d/sample-pdf-10/...


In [42]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [49]:
job = json_res
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://drive.google.com/file/d/sample-pdf-7/view'},
  {'links': 'https://drive.google.com/file/d/sample-pdf-8/view'}],
 [{'links': 'https://drive.google.com/file/d/sample-pdf-7/view'},
  {'links': 'https://drive.google.com/file/d/sample-pdf-8/view'}],
 [{'links': 'https://drive.google.com/file/d/sample-pdf-7/view'},
  {'links': 'https://drive.google.com/file/d/sample-pdf-8/view'}],
 [{'links': 'https://drive.google.com/file/d/sample-pdf-4/view'},
  {'links': 'https://drive.google.com/file/d/sample-pdf-14/view'}],
 [{'links': 'https://drive.google.com/file/d/sample-pdf-8/view'},
  {'links': 'https://drive.google.com/file/d/sample-pdf-7/view'}],
 [{'links': 'https://drive.google.com/file/d/sample-pdf-1/view'},
  {'links': 'https://drive.google.com/file/d/sample-pdf-2/view'}],
 [{'links': 'https://drive.google.com/file/d/sample-pdf-8/view'},
  {'links': 'https://drive.google.com/file/d/sample-pdf-11/view'}]]

In [50]:
job = json_res
job['skills']

['HTML',
 'CSS',
 'JavaScript',
 'Object-oriented programming language (e.g., JavaScript, Python, C#)',
 'Computer science fundamentals (e.g., data structures & algorithms, OO design)',
 'Front-end frameworks such as React, Angular',
 'UX best practices']

In [51]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Anhadpreet, a 4th Year Undergraduate at University of Alberta with major in computer science and minor in business.
        Over our experience, we have completed numerous projects with tailored solutions, fostering scalability, 
        process optimization, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Anhadpreet Singh 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Anhadpreet's portfolio: {link_list}
        Remember you are Anhadpreet, 4th Year undergraduate at University of Alberta. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Front-End Engineer Position

Dear Hiring Manager,

I am excited to apply for the Front-End Engineer position at your esteemed organization. As a 4th-year undergraduate student at the University of Alberta, majoring in Computer Science and minoring in Business, I am confident that my skills and experience make me an ideal candidate for this role.

Throughout my academic journey, I have developed a strong foundation in computer science fundamentals, including data structures, algorithms, and object-oriented design. I have also gained hands-on experience with front-end development technologies such as HTML, CSS, JavaScript, and React. My proficiency in these areas has enabled me to create scalable, efficient, and user-friendly software solutions.

In addition to my technical skills, I have a keen interest in UX best practices and have worked on various projects that demonstrate my ability to create intuitive and engaging user interfaces. My experience with the ful